# Un problema de regresión logística

En este laboratorio pondrás a prueba tus conocimientos sobre entrenamiento de modelos de clasificación utilizando regresión logística. Particularmente, realizarás los siguientes procedimientos:

1. Cargar un conjunto de datos.
2. Preparar los datos para el modelado.
3. Realizar la búsqueda de hiperparámetros para la regresión logística.
4. Evaluar los mejores modelos resultantes.

El problema a resolver es el siguiente: dados algunos indicadores de salud de una persona, queremos predecir si está en riesgo de padecer de una enfermedad cardiovascular o no. A lo largo del laboratorio encontrarás múltiples celdas con la siguiente estructura:
```python
#---------- Celda de pruebas - Ejercicio X.X. ----------
# Prueba 1
# Prueba 2
# ...
#-------------------------------------------------------
```
Estas celdas son utilizadas para calificar tus respuestas después de que realices tu envío, por lo que en ellas encontrarás los puntos específicos que serán evaluados. Asegúrate de no incluir código en estas celdas, ya que cualquier modificación será eliminada automáticamente al momento de realizar la evaluación. 

Antes de iniciar, vamos a importar las librerías necesarias:

In [2]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import make_pipeline

## 1. Carga de datos

Con las librerías importadas, realizaremos la carga del conjunto de datos:

### Ejercicio 1.1.

Utiliza Pandas para importar el archivo que contiene el conjunto de datos.

* La ruta del archivo .csv es: `./data/heart_disease_health_indicators.csv`, y ya se encuentra en el entorno de Coursera, solo debes importarlo.
* La variable resultante debe tener el nombre `data_raw`.
* **Formato esperado de respuesta:** se verificará que tu respuesta sea un `DataFrame` de Pandas con dimensiones `(253680,22)`.

In [3]:
ruta = './data/heart_disease_health_indicators.csv'
# your code here
data_raw = pd.read_csv(ruta)

In [4]:
#---------- Celda de Pruebas ----------
# El resultado existe
# El resultado es un DataFrame
# El resultado tiene las dimensiones correctas
#--------------------------------------

## 2. Preparación de datos

Primero vamos a definir la variable `data` para almacenar un conjunto de datos modificado:

In [5]:
data = data_raw.copy()

### Ejercicio 2.1.

Primero vas a realizar la limpieza de los datos. Utiliza Pandas para consultar si existen valores faltantes o filas duplicadas, y elimina estos datos.

* Utiliza funciones sobre el DataFrame para saber si contiene valores faltantes o filas duplicadas. (**Ejemplo: `data.<<Función>>`**)
* Utiliza las funciones de Pandas necesarias para eliminar valores faltantes y filas duplicadas. Asigna tu respuesta a la misma variable `data`. (**Ejemplo: `data = data.<<Función>>`**)
* Encontrarás la línea `data` al final de la celda. Esta línea se usa para que puedas visualizar el resultado, por lo que debes dejarla al final y no debes modificarla.
* **Formato esperado de respuesta:** se verificará que `data` sea un `DataFrame` de Pandas con dimensiones `(229781,22)`. La variable `data` no debe contener valores faltantes ni filas duplicadas para que tu respuesta sea correcta.

In [7]:
# Verificar y eliminar valores faltantes
if data_raw.isnull().any().any():
    data_raw = data_raw.dropna()

# Verificar y eliminar filas duplicadas
if data_raw.duplicated().any():
    data_raw = data_raw.drop_duplicates()

# Renombrar el DataFrame a data para seguir con las instrucciones
data = data_raw

# Mostrar las dimensiones del DataFrame para verificar
print(data.shape)

(229781, 22)


In [8]:
#---------- Celda de Pruebas ----------
# La variable 'data' existe
# La variable 'data' es un DataFrame
# La variable 'data' no contiene valores faltantes
# La variable 'data' no contiene filas duplicadas
#--------------------------------------

### Ejercicio 2.2.

Ahora debes dividir el conjunto de datos en entrenamiento y pruebas. Usando el 80% de los datos para entrenar el modelo y el 20% restante para probarlo, utiliza `scikit-learn` para separar el conjunto de datos en dos.

* Guarda tu respuesta en dos variables: `train` y `test`. (**Ejemplo: `train, test = <<Función>>`**)
* Utiliza el parámetro `random_state=0`. Esto hará que la partición sea siempre la misma.
* Encontrarás la línea `train.head()`. Esta línea se usa para que puedas visualizar el resultado del conjunto de entrenamiento. Déjala al final de la celda y no la modifiques.
* **Formato esperado de respuesta:** se verificará que tu respuesta sean dos variables de tipo `pd.DataFrame`.
    * `train` debe tener dimensiones `(183824,22)`.
    * `test` debe tener dimensiones `(45957,22)`.

In [9]:
from sklearn.model_selection import train_test_split

# Proporción de división: 80% para entrenamiento, 20% para pruebas
proporcion_entrenamiento = 0.8

# Dividir los datos en conjuntos de entrenamiento y pruebas
train, test = train_test_split(data, test_size=1 - proporcion_entrenamiento, random_state=0)

# Mostrar las dimensiones para verificar
print("Dimensiones del conjunto de entrenamiento:", train.shape)
print("Dimensiones del conjunto de pruebas:", test.shape)

# Visualización del conjunto de entrenamiento (según lo solicitado)
train.head()

Dimensiones del conjunto de entrenamiento: (183824, 22)
Dimensiones del conjunto de pruebas: (45957, 22)


,HeartDiseaseorAttack,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,Diabetes,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
115369,0.0,0.0,1.0,0.0,24.0,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,1.0,2.0,0.0,0.0,0.0,6.0,6.0,8.0
145209,0.0,0.0,0.0,1.0,29.0,1.0,0.0,0.0,1.0,1.0,...,0.0,1.0,3.0,2.0,3.0,0.0,0.0,2.0,5.0,4.0
241128,0.0,0.0,0.0,1.0,28.0,0.0,0.0,2.0,0.0,1.0,...,1.0,0.0,3.0,0.0,0.0,1.0,1.0,12.0,4.0,4.0
72372,0.0,1.0,0.0,1.0,32.0,1.0,1.0,0.0,0.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,1.0,10.0,4.0,5.0
236231,0.0,0.0,1.0,1.0,37.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,3.0,10.0,0.0,0.0,0.0,7.0,6.0,8.0


In [10]:
#---------- Celda de Pruebas ----------
# Las variables "train" y "test" existen
# Las variables "train" y "test" son un DataFrame
# Las variables tienen las dimensiones correctas
#--------------------------------------

### Ejercicio 2.3.

Ahora debes aislar la variable objetivo, `HeartDiseaseorAttack`, de las variables independientes.

* Crea una variable con nombre `x_train` y asígnale la operación necesaria para almacenar solo las variables descriptoras del conjunto de entrenamiento.
* Crea una variable con nombre `y_train` y asígnale la operación necesaria para almacenar la variable objetivo del conjunto de entrenamiento.
* **Formato esperado de respuesta:** se verificará que tu respuesta sean dos variables. 
    * `x_train` debe ser de tipo `pd.DataFrame` y tener dimensiones `(183824,21)`.
    * `y_train` debe ser de tipo `pd.Series` y tener dimensiones `(183824,)`.

In [11]:
# Crear x_train excluyendo la columna HeartDiseaseorAttack
x_train = train.drop('HeartDiseaseorAttack', axis=1)

# Crear y_train con solo la columna HeartDiseaseorAttack
y_train = train['HeartDiseaseorAttack']

# Mostrar las dimensiones para verificar
print("Dimensiones de x_train:", x_train.shape)
print("Dimensiones de y_train:", y_train.shape)

Dimensiones de x_train: (183824, 21)
Dimensiones de y_train: (183824,)


In [12]:
#---------- Celda de Pruebas ----------
# Las variables "x_train" y "y_train" existen
# La variable "x_train" es un DataFrame
# La variable "y_train" es una Serie de Pandas
# Las variables tienen las dimensiones correctas
#--------------------------------------

### Ejercicio 2.4.

El siguiente paso es realizar la estandarización de nuestros datos. Inicialmente, vamos a almacenar los nombres de las columnas del conjunto de entrenamiento en la variable `columns`:

In [13]:
columns = x_train.columns

Adicionalmente, definiremos un objeto de la clase `StandardScaler()`, que utilizarás para realizar la estandarización:

In [14]:
scaler = StandardScaler()

Con las variables definidas, realiza la estandarización de los datos. Modifica el conjunto de variables independientes, `x_train`, con las operaciones necesarias para estandarizarlo. Particularmente, deberás seguir estos pasos:

* Utiliza la variable `scaler` para transformar los datos de `x_train`. Asigna el resultado a la misma variable sobreescribiendo su contenido. (**Ejemplo: `x_train = scaler.<<Función>>`**)
* Reconstruye el DataFrame de variables independientes utilizando la información almacenada en `columns`. Asigna el resultado a `x_train`. (**Ejemplo: `x_train = <<Función>>`**)
* **Formato esperado de respuesta:** se verificará que `x_train` sea un `DataFrame` de Pandas con dimensiones `(183824,21)`. Los valores de `x_train` deben estar modificados por el `scaler`.

In [15]:
from sklearn.preprocessing import StandardScaler

# Almacenar los nombres de las columnas
columns = x_train.columns

# Definir el objeto scaler
scaler = StandardScaler()

# Ajustar y transformar x_train
x_train_scaled = scaler.fit_transform(x_train)

# Reconstruir el DataFrame con las columnas originales
x_train = pd.DataFrame(x_train_scaled, columns=columns)

# Mostrar las dimensiones para verificar
print("Dimensiones de x_train después de la estandarización:", x_train.shape)

Dimensiones de x_train después de la estandarización: (183824, 21)


In [16]:
#---------- Celda de Pruebas ----------
# La variable "x_train" es un DataFrame
# La variable tiene las dimensiones correctas
#--------------------------------------

## 3. Búsqueda de hiperparámetros y entrenamiento del modelo

Con el conjunto de datos preparado, es momento de entrenar el modelo de regresión logística. Primero vamos a definir un objeto de la clase `KFold`, con el que definiremos 10 subconjuntos sobre el conjunto de entrenamiento:

In [ ]:
kfold = KFold(n_splits=10, shuffle=True, random_state = 0)

También definiremos el objeto de regresión logística:

In [ ]:
reglog = LogisticRegression()

### Ejercicio 3.1.

El siguiente paso es definir el espacio de búsqueda de los hiperparámetros.

* Define una variable con el nombre `param_grid` y asígnale la expresión necesaria para crear un diccionario con dos tuplas (**Ejemplo: `param_grid = <<Expresión>>`**):
    * Llave `C` y valor `valores_C`.
    * Llave `solver` y valor `valores_solver`.
* **Formato esperado de respuesta:** se verificará que `param_grid` sea un diccionario (`dict`) de Python con dos posiciones. Las tuplas deben tener las llaves descritas anteriormente y sus valores deben ser iguales a las variables definidas (`valores_C` y `valores_solver`), en ese orden.

In [17]:
valores_C = [0.1, 0.5]
valores_solver = ['newton-cg', 'liblinear']

param_grid = {
    'C': valores_C,
    'solver': valores_solver
}

In [18]:
#---------- Celda de Pruebas ----------
# La variable "param_grid" existe
# La variable "param_grid" es un diccionario
# La variable tiene la longitud correcta
#--------------------------------------

### Ejercicio 3.2.

Finalmente, debes crear el objeto de tipo `GridSearchCV` para realizar la búsqueda. Utiliza las variables `reglog`, `param_grid` y `kfold` para definirlo:

* Define una variable con el nombre `grid` y asígnale la función necesaria para crear un objeto de la clase `GridSearchCV`. (**Ejemplo: `grid = <<Función>>`**)
* Utiliza el parámetro `scoring='accuracy'` para que se seleccione el mejor modelo de acuerdo con los valores de exactitud.
* Si quieres, puedes utilizar el parámetro `verbose=3` para que cada paso de la búsqueda se muestre en consola.
* **Formato esperado de respuesta:** se verificará que `grid` sea de tipo `GridSearchCV`. El atributo `scoring` debe ser `'accuracy'` para que tu respuesta sea correcta.

In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, KFold

# Definir el modelo de regresión logística
reglog = LogisticRegression()

# Definir el espacio de búsqueda de hiperparámetros
valores_C = [0.1, 0.5]
valores_solver = ['newton-cg', 'liblinear']
param_grid = {
    'C': valores_C,
    'solver': valores_solver
}

# Definir el método de validación cruzada
kfold = KFold(n_splits=10, shuffle=True, random_state=0)

# Crear el objeto GridSearchCV
grid = GridSearchCV(estimator=reglog, param_grid=param_grid, cv=kfold, scoring='accuracy', verbose=3)

# Nota: Ahora puedes proceder con la búsqueda de hiperparámetros utilizando grid.fit(x_train, y_train), etc.

In [21]:
#---------- Celda de Pruebas ----------
# La variable "grid" existe
# La variable "grid" es un objeto de la clase GridSearchCV
# La variable "grid" usa la exactitud como método de selección
#--------------------------------------

### Ejercicio 3.3.

A continuación, realiza la búsqueda de hiperparámetros utilizando el conjunto de entrenamiento, compuesto por las variables `x_train` y `y_train`.

* Para este ejercicio no debes asignar tu resultado a ninguna variable. Es decir, solo debes ejecutar una función sobre la variable `grid`, utilizando las variables `x_train` y `y_train` como parámetros. (**Ejemplo: `grid.<<Función>>`**)
* **Formato esperado de respuesta:** se verificará que el objeto `grid` tenga definidos los atributos `best_params_` y `best_estimator_`.
    * `grid.best_params_` debe ser **exactamente igual** a 2.
    * `grid.best_estimator_` debe ser de tipo `LogisticRegression`.

In [22]:
# Realizar la búsqueda de hiperparámetros utilizando GridSearchCV
grid.fit(x_train, y_train)

Fitting 10 folds for each of 4 candidates, totalling 40 fits
[CV] C=0.1, solver=newton-cg .........................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ............. C=0.1, solver=newton-cg, score=0.898, total=  12.3s
[CV] C=0.1, solver=newton-cg .........................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   12.3s remaining:    0.0s


[CV] ............. C=0.1, solver=newton-cg, score=0.901, total=  12.3s
[CV] C=0.1, solver=newton-cg .........................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   24.7s remaining:    0.0s


[CV] ............. C=0.1, solver=newton-cg, score=0.896, total=  11.8s
[CV] C=0.1, solver=newton-cg .........................................
[CV] ............. C=0.1, solver=newton-cg, score=0.902, total=  11.6s
[CV] C=0.1, solver=newton-cg .........................................
[CV] ............. C=0.1, solver=newton-cg, score=0.901, total=  12.7s
[CV] C=0.1, solver=newton-cg .........................................
[CV] ............. C=0.1, solver=newton-cg, score=0.899, total=  14.1s
[CV] C=0.1, solver=newton-cg .........................................
[CV] ............. C=0.1, solver=newton-cg, score=0.898, total=  12.2s
[CV] C=0.1, solver=newton-cg .........................................
[CV] ............. C=0.1, solver=newton-cg, score=0.899, total=  12.6s
[CV] C=0.1, solver=newton-cg .........................................
[CV] ............. C=0.1, solver=newton-cg, score=0.903, total=  12.0s
[CV] C=0.1, solver=newton-cg .........................................
[CV] .

[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:  4.6min finished


GridSearchCV(cv=KFold(n_splits=10, random_state=0, shuffle=True),
             error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [0.1, 0.5], 'solver': ['newton-cg', 'liblinear']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=3)

In [26]:
#---------- Celda de Pruebas ----------
# El atributo "best_params_" de la variable "grid" existe
# El atributo "best_estimator_" de la variable "grid" existe
#--------------------------------------

Podemos observar los mejores hiperparámetros usando el atributo `best_params_`:

In [27]:
print("Mejores parámetros: ", grid.best_params_)

Mejores parámetros:  {'C': 0.1, 'solver': 'liblinear'}


Además, vamos a definir la variable `mejor_modelo` para realizar las pruebas:

In [28]:
mejor_modelo = grid.best_estimator_

## 4. Evaluación del modelo

Por último, evaluarás el modelo entrenado utilizando el conjunto de pruebas.

### Ejercicio 4.1.

Separa las variables independientes y la variable objetivo en el conjunto de pruebas. Para ello, utiliza Pandas para crear dos variables, `x_test` y `y_test`, que almacenarán las variables independientes y la variable objetivo, respectivamente.

* Crea una variable con nombre `x_test` y asígnale la operación necesaria para almacenar solo las variables independientes del conjunto de pruebas. (**Ejemplo: `x_test = test.<<Función>>`**)
* Crea una variable con nombre `y_test` y asígnale la operación necesaria para almacenar la variable objetivo del conjunto de pruebas. (**Ejemplo: `y_test = <<Consulta>>`**)
* **Formato esperado de respuesta:** se verificará que tu respuesta sean dos variables. 
    * `x_test` debe ser de tipo `pd.DataFrame` y tener dimensiones `(45957,21)`.
    * `y_test` debe ser de tipo `pd.Series` y tener dimensiones `(45957,)`.

In [29]:
# Crear x_test excluyendo la columna HeartDiseaseorAttack
x_test = test.drop('HeartDiseaseorAttack', axis=1)

# Crear y_test con solo la columna HeartDiseaseorAttack
y_test = test['HeartDiseaseorAttack']

In [ ]:
#---------- Celda de Pruebas ----------
# Las variables existen
# La variable "x_test" es un DataFrame
# La variable "y_test" es una Serie de Pandas
# Las variables tienen las dimensiones correctas
#--------------------------------------

### Ejercicio 4.2.

Realiza la estandarización de los datos del conjunto de pruebas. Modifica el conjunto de variables independientes, `x_test`, con las operaciones necesarias para estandarizarlo. Particularmente, deberás seguir estos pasos:

* Utiliza la variable `scaler` para transformar los datos de `x_test` **utilizando solamente la información del conjunto de entrenamiento**. Asigna el resultado a la misma variable, sobreescribiendo su contenido. (**Ejemplo: `x_test = scaler.<<Función>>`**)
* Reconstruye el DataFrame de variables independientes utilizando la información almacenada en `columns`. Asigna el resultado a `x_test`. (**Ejemplo: `x_test = <<Función>>`**)
* **Formato esperado de respuesta:** se verificará que `x_test` sea un `DataFrame` de Pandas con dimensiones `(45957,21)`. Los valores de `x_test` deben estar modificados por el `scaler`.

In [30]:
# Transformar x_test utilizando el scaler ajustado con x_train
x_test_scaled = scaler.transform(x_test)

# Reconstruir el DataFrame con las columnas originales
x_test = pd.DataFrame(x_test_scaled, columns=columns)

In [31]:
#---------- Celda de Pruebas ----------
# La variable "x_test" es un DataFrame
# La variable tiene las dimensiones correctas
#--------------------------------------

### Ejercicio 4.3.

Con el conjunto de pruebas preparado, realiza predicciones con el fin de compararlas con los valores reales almacenados en `y_test`.

* Utiliza la variable `mejor_modelo` para realizar las predicciones sobre el mejor modelo. Asigna el resultado a una variable con nombre `y_pred` (**Ejemplo: `y_pred = mejor_modelo.<<Función>>`**).
* **Formato esperado de respuesta:** se verificará que `y_pred` sea de tipo `np.ndarray` y tenga dimensiones `(45957,)`.

In [32]:
# Obtener el mejor modelo del objeto grid
mejor_modelo = grid.best_estimator_

# Realizar predicciones sobre el conjunto de pruebas
y_pred = mejor_modelo.predict(x_test)

In [33]:
#---------- Celda de Pruebas ----------
# La variable "y_pred" existe
# La variable "y_pred" es un arreglo
# La variable "y_pred" tiene las dimensiones correctas
#--------------------------------------

### Ejercicio 4.4.

Utiliza el conjunto de predicciones `y_pred` y el conjunto de valores reales `y_test` para obtener la matriz de confusión del mejor modelo.

* Haz un llamado a la función que retorna la matriz de confusión como un arreglo. Asigna el resultado a una nueva variable con el nombre `p44` (**Ejemplo: `p44 = <<Función>>`**).
* Encontrarás la línea `p44` al final de la celda. Esta línea se usa para que puedas visualizar la matriz resultante, por lo que no la debes modificar.
* **Formato esperado de respuesta:** se verificará que `p44` sea de tipo `np.ndarray` y tenga dimensiones `(2,2)`.
    * El número de verdaderos negativos debe estar entre 40000 y 41000.
    * El número de verdaderos positivos debe estar entre 550 y 650.
    * El número de falsos negativos debe estar entre 450 y 550.
    * El número de falsos positivos debe estar entre 4000 y 4500.

In [34]:
from sklearn.metrics import confusion_matrix

# Obtener la matriz de confusión
p44 = confusion_matrix(y_test, y_pred)

In [35]:
#---------- Celda de Pruebas ----------
# La variable "p44" existe
# La variable "p44" es un arreglo de numpy
# La variable "p44" tiene las dimensiones correctas
#--------------------------------------

### Ejercicio 4.5.

Finalmente, utiliza `scikit-learn` para obtener cuatro métricas de rendimiento sobre las predicciones del modelo:

* Define una variable con el nombre `accuracy` y asígnale la función necesaria para obtener la exactitud (`accuracy = <<Función>>`).
* Define una variable con el nombre `recall` y asígnale la función necesaria para obtener la sensibilidad (`recall = <<Función>>`).
* Define una variable con el nombre `precision` y asígnale la función necesaria para obtener la precisión (`precision = <<Función>>`).
* Define una variable con el nombre `f1_score` y asígnale la función necesaria para obtener el F1-Score (`f1_score = <<Función>>`).
* **Formato esperado de respuesta:** se verificará que las cuatro variables sean de tipo `np.float64` o de tipo `float` de Python. Todas las métricas deben estar entre 0 y 1.

In [36]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# Calcular la exactitud (accuracy)
accuracy = accuracy_score(y_test, y_pred)

# Calcular la sensibilidad (recall)
recall = recall_score(y_test, y_pred)

# Calcular la precisión (precision)
precision = precision_score(y_test, y_pred)

# Calcular el F1-Score
f1_score = f1_score(y_test, y_pred)

print('-'*10+"Regresión logística"+'-'*10)
print("Exactitud: ", accuracy)
print("Sensibilidad: ", recall)
print('Precisión: ', precision)
print('F1-Score: ', f1_score)

----------Regresión logística----------
Exactitud:  0.8962726026503035
Sensibilidad:  0.1236880016464293
Precisión:  0.5414414414414415
F1-Score:  0.20137376444965654


In [37]:
#---------- Celda de Pruebas ----------
# Las variables "accuracy", "recall", "precision" y "f1_score" existen
# Las variables son números
# Las variables tienen valores válidos
#--------------------------------------